# Prepairation

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/deepLearningProject/*'
!cp -r $rootDir .



In [ ]:
!pip install --upgrade --no-cache-dir gdown 
! pip install insightface
! pip install onnxruntime-gpu

# 1-1 face detection 

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.models import resnet50, ResNet50_Weights

from libs.MSCTDdataset import MSCTD
from libs.transforms import Compose,toTensor,resize

import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from libs.faceExtractor import faceExtractor
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report





In [ ]:
data_transforms =Compose([ToTensor(),Resize((640, 640)),])
training_data = MSCTD(
        mode='train', 
        transformer=data_transforms,
        download=True,
        root_dir=".",
        read_mode="single")

valid_data = MSCTD(
        mode='validation',
        transformer=data_transforms,
        download=True,
        root_dir=".",
        read_mode="single")

test_data = MSCTD(
        mode='test',
        transformer=data_transforms,
        download=True,
        root_dir=".",
        read_mode="single")

 ابولفصل تمام سلول های این بخش بدون آگمنتیشن هستند

In [ ]:
face_train=faceExtractor(rootDir=".",mode="train")
face_train.run(training_data)
print(f"[train] number of images that has no detected face in it :{face_train.fault}")

In [ ]:
face_validation=faceExtractor(rootDir=".",mode="validation")
face_validation.run(valid_data)
print(f"[validation] number of images that has no detected face in it :{face_validation.fault}")

In [ ]:
face_test=faceExtractor(rootDir=".",mode="test")
face_test.run(test_data)
print(f"[test] number of images that has no detected face in it :{face_validation.fault}")

# 1-2 Learning an Network on face (DAN)

In [ ]:
# get image from Drive (ابولفصل باید این رو دانلود کنند به نظرم کد دانلود را همین جا بگذار که دانلود و از زیپ خارج کنند)
rootDir = '/content/drive/MyDrive/FaceDataset.zip'
!unzip $rootDir -d .


In [ ]:
# training
!python faceSentiment.py "--Facedataset_path=."  "--workers=2"  "--num_head=4" "--Test=0" "--model_path=/content/checkpoints/rafdb_epoch12_acc0.3844_bacc0.3561.pth"

In [ ]:
# test
!python faceSentiment.py "--Facedataset_path=."  "--workers=2"  "--num_head=4" "--Test=1" "--model_path=/content/checkpoints/rafdb_epoch12_acc0.3844_bacc0.3561.pth"

# 1-3 Join Networks and get final result 

In [ ]:
tf = Compose([]) 
training_data = MSCTD(
        mode='train', 
        transformer=tf,
        download=True,
        root_dir=".",
        read_mode="scene")

valid_data = MSCTD(
        mode='validation',
        transformer=tf,
        download=True,
        root_dir=".",
        read_mode="scene")

test_data = MSCTD(
        mode='test',
        transformer=tf,
        download=True,
        root_dir=".",
        read_mode="scene")


ابولفضل تمام این قسمت احتیاج به دیتا ست دارد و البته تمام تصاویر تولید شده از قبل البته بدون آگمنیشن

In [ ]:
# train
!python ModelPip.py "--FaceDataset_path=." "--Test=0" "--lr=1e-3" "--workers=2" "--batch_size=256" "--epochs=40" "--num_head=4" "--Model_path=/content/checkpoints/rafdb_epoch18_acc0.3992_bacc0.3682.pth"

In [ ]:
# test
!python ModelPip.py "--FaceDataset_path=." "--Test=1" "--lr=1e-3" "--workers=2" "--batch_size=256" "--epochs=40" "--num_head=4" "--Model_path=/content/checkpoints/rafdb_epoch18_acc0.3992_bacc0.3682.pth"